In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('C:/Users/BABU/Downloads/all/Breast_cancer_data.csv')

In [3]:
df

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [5]:
df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


In [6]:
df.corr()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
mean_radius,1.000000,0.323782,0.997855,0.987357,0.170581,-0.730029
mean_texture,0.323782,1.000000,0.329533,0.321086,-0.023389,-0.415185
mean_perimeter,0.997855,0.329533,1.000000,0.986507,0.207278,-0.742636
mean_area,0.987357,0.321086,0.986507,1.000000,0.177028,-0.708984
mean_smoothness,0.170581,-0.023389,0.207278,0.177028,1.000000,-0.358560
diagnosis,-0.730029,-0.415185,-0.742636,-0.708984,-0.358560,1.000000


In [7]:
# there are no null values and outliers

In [8]:
df['diagnosis'].value_counts()

1    357
0    212
Name: diagnosis, dtype: int64

In [9]:
x=df.drop(['diagnosis'],axis=1)
y=df['diagnosis']

In [10]:
#model Creation

In [11]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,f1_score,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV,cross_val_score,RepeatedStratifiedKFold,RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

adc=AdaBoostClassifier()
rf=RandomForestClassifier()
dt=DecisionTreeClassifier()
knn=KNeighborsClassifier()
sv=SVC()
xgb=XGBClassifier(eval_metric='logloss')
lr=LogisticRegression()
cv=RepeatedStratifiedKFold(n_splits=10,n_repeats=3)

In [12]:
for model in [adc,rf,dt,sv,knn,xgb,lr]:
    print(cross_val_score(model,x,y,cv=cv,scoring='accuracy').mean())

0.9297305764411028
0.9255847953216372
0.9016186299081036
0.8863826232247285
0.8881474519632414
0.9232873851294905
0.9085526315789475


In [13]:
# Adaboost,Randomforest and xgboost are best for this dataset
#already we use randomforest for other disease prediction
# so we go for AdaBoost Classifier

In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10,stratify=y)
from collections import Counter
Counter(y_train)

Counter({1: 285, 0: 170})

In [15]:
#handling imbalanced datset
#use smote technique to create synthetic samples from the minor class in x_train,y_train
from imblearn.over_sampling import SMOTE
smote=SMOTE()
x_train_balanced,y_train_balanced=smote.fit_resample(x_train,y_train)
from collections import Counter
Counter(y_train_balanced)

Counter({1: 285, 0: 285})

In [16]:
final_model=AdaBoostClassifier(random_state=10)
final_model.fit(x_train_balanced,y_train_balanced)

AdaBoostClassifier(random_state=10)

In [17]:
#testing
y_pred=final_model.predict(x_test)
print('Accuracy Score   :',accuracy_score(y_test,y_pred))
print('Confusion Matrix :')
print(confusion_matrix(y_test,y_pred))
print('Recall Score     :',recall_score(y_test,y_pred))

Accuracy Score   : 0.8859649122807017
Confusion Matrix :
[[34  8]
 [ 5 67]]
Recall Score     : 0.9305555555555556


In [18]:
#we got very good accuracy and recall score
#so no need to tuning the parameter 

In [21]:
#testing
final_model.predict([[7.76,24.54,47.92,181,0.0526]])[0]

1

In [22]:
#Model file creation using joblib

In [23]:
import joblib
joblib.dump(final_model,'final_model_breastcancer.joblib')

['final_model_breastcancer.joblib']

In [25]:
#using model file to predict values
model=joblib.load('final_model_breastcancer.joblib')
model.predict([[7.76,24.54,47.92,181,0.0526]])[0]

1

In [26]:
#succeed